In [ ]:
from aiida import load_profile

load_profile();

# AiiDAlab Code Setup

This page provides an interface for creating AiiDA `Code` references to executable on 
an AiiDA  `Computer` node.

In [ ]:
%%html

<style>
  .output_subarea {
    max-width: none !important;
  }
  .header-tr {
    column-gap: 5px;
  }
  .th {
    padding: 2px 5px;
    font-weight: bold;
    color: #fff;
    text-align: center;
    background-color: #000;
  }
  .tr:hover {
    background-color: #f5f5f5;
  }
</style>

## Create new code

In [ ]:
import ipywidgets as ipw
from aiidalab_widgets_base.computational_resources import _ResourceSetupBaseWidget
from aiidalab_widgets_base.utils import StatusHTML

setup_message = StatusHTML(clear_after=15)

widget = _ResourceSetupBaseWidget()
ipw.dlink(
    (widget, "message"),
    (setup_message, "message"),
)

ipw.VBox(
    children=[
        widget,
        setup_message,
    ]
)

## Available codes

In [ ]:
from home.code_setup import create_paginated_table, fetch_code_data

output = ipw.Output()


def render():
    data = fetch_code_data()
    paginated_table = create_paginated_table(data)
    output.clear_output()
    with output:
        display(paginated_table)


display(output)
render()

In [ ]:
widget.quick_setup_button.on_click(lambda _: render())